In [1]:

from PyQt5.QtWidgets import QMainWindow, QApplication, QLineEdit, QComboBox, QDialogButtonBox, QLabel, QDialog, QPushButton, QFileDialog
from PyQt5 import uic
import sys
import all_funcs
from math import ceil
import os
import csv
from nd2reader import ND2Reader
import matplotlib.pyplot as plt
from skimage import data
from skimage.data import astronaut
import cv2
import numpy as np
from tifffile import imwrite
from nd2reader import ND2Reader
import numpy as np
import glob
import csv
import os
from all_funcs import extract_roi_coordinates
from all_funcs import cell_size_extractor
from all_funcs import clean_roi_borders
from all_funcs import pre_clean
from all_funcs import noisify
from all_funcs import crop_using_roi_tuple
from skimage.measure import label, regionprops, regionprops_table
import math
import pickle

In [2]:

output_path="/Users/saranshumale/Documents/Data/MitoNetwork_Project/AgarPadAnalysis/Nov25_29_acdc/agarpad_mito_timelapse_analysis_acdc/Gal/Output/"
roi_path = "/Users/saranshumale/Documents/Data/MitoNetwork_Project/AgarPadAnalysis/Nov25_29_acdc/agarpad_mito_timelapse_analysis_acdc/Gal/"


### Run the script below to crop and extract microcolony TIFs from the raw files

In [5]:
#### Run this to go through ROIs and then save the BF2 images
## -> Raw Image
#           BF2 - create a folder for each ROI

##Prime95B
##Channel001 is the BF channel

raw_img_path = '/Volumes/Aman_DATA/Agarosepad/gal_nov29_ylb2128/Pos1/'

bf_output_path = "/Users/saranshumale/Documents/Data/MitoNetwork_Project/AgarPadAnalysis/Nov25_29_acdc/agarpad_mito_timelapse_analysis_acdc/Gal/Output/BF2/"

roi_tuples = extract_roi_coordinates(roi_path+'fov1'+'.zip')
l = len(roi_tuples)

for j in range(l):
    
    os.mkdir(bf_output_path+"cell_"+str(j))

    roi_coordinates = roi_tuples[j][0]

    for time in range(70):
        if time <10:
            bf_filename = "img_channel001_position001_time00000000"+str(time)+"_z017.tif"
        else:
            bf_filename = "img_channel001_position001_time0000000"+str(time)+"_z017.tif"
        
        bf_image = cv2.imread(raw_img_path+bf_filename,-1)

        bf_cropped = np.array(crop_using_roi_tuple(roi_coordinates, np.array(bf_image)))

        imwrite(bf_output_path+"cell_"+str(j)+"/"+"pos1_BF_"+str(time)+".tif",bf_cropped)



In [8]:
#### Run this to go through ROIs and then save the BF2 and Mito files in separate folders
## -> Raw Image
#           Mito - create a folder for each ROI       
##Prime95B
##Channel000 is the Mito channel

raw_img_path = '/Volumes/Aman_DATA/Agarosepad/gal_nov29_ylb2128/Pos1/'

mito_output_path = "/Users/saranshumale/Documents/Data/MitoNetwork_Project/AgarPadAnalysis/Nov25_29_acdc/agarpad_mito_timelapse_analysis_acdc/Gal/Output/Mito/"


roi_tuples = extract_roi_coordinates(roi_path+'fov1'+'.zip')
l = len(roi_tuples)

for j in range(l):
    os.mkdir(mito_output_path+"cell_"+str(j))

    roi_coordinates = roi_tuples[j][0]

    for time in range(70):
        mito_stack = np.empty((34,roi_tuples[j][0][3]-roi_tuples[j][0][1],roi_tuples[j][0][2]-roi_tuples[j][0][0]))
        if time <10:
            for z in range(10):
                mito_filename = "img_channel000_position001_time00000000"+str(time)+"_z00"+str(z)+".tif"
                mito_img = cv2.imread(raw_img_path+mito_filename,-1)
                mito_cropped = np.array(crop_using_roi_tuple(roi_coordinates, np.array(mito_img)))
                mito_stack[z] = mito_cropped
            for z in range(10,34):
                mito_filename = "img_channel000_position001_time00000000"+str(time)+"_z0"+str(z)+".tif"
                mito_img = cv2.imread(raw_img_path+mito_filename,-1)
                mito_cropped = np.array(crop_using_roi_tuple(roi_coordinates, np.array(mito_img)))
                mito_stack[z] = mito_cropped


        else:

            for z in range(10):
                mito_filename = "img_channel000_position001_time0000000"+str(time)+"_z00"+str(z)+".tif"
                mito_img = cv2.imread(raw_img_path+mito_filename,-1)
                mito_cropped = np.array(crop_using_roi_tuple(roi_coordinates, np.array(mito_img)))
                mito_stack[z] = mito_cropped
            for z in range(10,34):
                mito_filename = "img_channel000_position001_time0000000"+str(time)+"_z0"+str(z)+".tif"
                mito_img = cv2.imread(raw_img_path+mito_filename,-1)
                mito_cropped = np.array(crop_using_roi_tuple(roi_coordinates, np.array(mito_img)))
                mito_stack[z] = mito_cropped
        


        imwrite(mito_output_path+"cell_"+str(j)+"/"+"pos1_mito_"+str(time)+".tif",mito_stack.astype('uint16'),dtype='uint16')


In [9]:
import os

def remove_mask_from_filenames(folder_path):
    # Iterate through all files in the specified folder
    for filename in os.listdir(folder_path):
        # Check if "_mask" is in the filename
        if "_mask" in filename:
            # Create the new filename by removing "_mask"
            new_filename = filename.replace("_mask", "")
            # Generate the full path for old and new filenames
            old_file = os.path.join(folder_path, filename)
            new_file = os.path.join(folder_path, new_filename)
            # Rename the file
            os.rename(old_file, new_file)
            print(f'Renamed: "{filename}" to "{new_filename}"')

# Specify the folder path
folder_path = '/path/to/your/folder'  # Change this to your folder path

# Call the function to remove "_mask" from filenames
remove_mask_from_filenames("/Users/saranshumale/Documents/Data/MitoNetwork_Project/AgarPadAnalysis/Nov25_29_acdc/agarpad_mito_timelapse_analysis_acdc/Gal/Output/BF2/cell_0_masks/")
remove_mask_from_filenames("/Users/saranshumale/Documents/Data/MitoNetwork_Project/AgarPadAnalysis/Nov25_29_acdc/agarpad_mito_timelapse_analysis_acdc/Gal/Output/BF2/cell_1_masks/")



Renamed: "pos1_BF_41_mask.tif" to "pos1_BF_41.tif"
Renamed: "pos1_BF_40_mask.tif" to "pos1_BF_40.tif"
Renamed: "pos1_BF_36_mask.tif" to "pos1_BF_36.tif"
Renamed: "pos1_BF_37_mask.tif" to "pos1_BF_37.tif"
Renamed: "pos1_BF_56_mask.tif" to "pos1_BF_56.tif"
Renamed: "pos1_BF_57_mask.tif" to "pos1_BF_57.tif"
Renamed: "pos1_BF_68_mask.tif" to "pos1_BF_68.tif"
Renamed: "pos1_BF_69_mask.tif" to "pos1_BF_69.tif"
Renamed: "pos1_BF_0_mask.tif" to "pos1_BF_0.tif"
Renamed: "pos1_BF_1_mask.tif" to "pos1_BF_1.tif"
Renamed: "pos1_BF_21_mask.tif" to "pos1_BF_21.tif"
Renamed: "pos1_BF_20_mask.tif" to "pos1_BF_20.tif"
Renamed: "pos1_BF_15_mask.tif" to "pos1_BF_15.tif"
Renamed: "pos1_BF_14_mask.tif" to "pos1_BF_14.tif"
Renamed: "pos1_BF_62_mask.tif" to "pos1_BF_62.tif"
Renamed: "pos1_BF_63_mask.tif" to "pos1_BF_63.tif"
Renamed: "pos1_BF_51_mask.tif" to "pos1_BF_51.tif"
Renamed: "pos1_BF_50_mask.tif" to "pos1_BF_50.tif"
Renamed: "pos1_BF_18_mask.tif" to "pos1_BF_18.tif"
Renamed: "pos1_BF_19_mask.tif" to "